<a href="https://colab.research.google.com/github/Mohammad-H-Zahrawi/Challenges-Kaggle/blob/main/Twitter%20Sentiment%20Analysis-Batch/clusterFinal2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words('english'))


In [ ]:
newStopWords = {'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'd',
 'did',
 'do',
 'does',
 'doing',
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 'has',
 'have',
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 'more',
 'most',
 'my',
 'myself',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'she',
 "she's",
 'should',
 "should've",
 'shouldn',
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'wasn',
 'we',
 'were',
 'what',
 'when',
 'where',
 'which',
 'while',
 'who',
 'whom',
 'why',
 'will',
 'with',
 'won',
 'y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.metrics import f1_score
from sklearn.cluster import KMeans


# Load data
df=pd.read_csv('/content/drive/MyDrive/twitter/train1.csv', encoding='ISO-8859-1')
df2=pd.read_csv('/content/drive/MyDrive/twitter/test1.csv', encoding='ISO-8859-1')

# df = pd.read_csv("train1.csv", encoding='ISO-8859-1')
# df2 = pd.read_csv("test1.csv", encoding='ISO-8859-1')
test_ids = df2['id']
X_test = df2.copy()
df = df.drop(columns=['id', 'user', 'date', 'flag'])
df2 = df2.drop(columns=['id', 'user', 'date', 'flag'])

# Text preprocessing functions
from bs4 import BeautifulSoup

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


def remove_urls(text):
    clean_text = re.sub(r'http\S+', '', text)
    return clean_text

def convert_to_lowercase(text):
    return text.lower()

def replace_chat_words(text):
    chat_words = {
        "BRB": "Be right back",
        "BTW": "By the way",
        "OMG": "Oh my God/goodness",
        "TTYL": "Talk to you later",
        "OMW": "On my way",
        "SMH/SMDH": "Shaking my head/shaking my darn head",
        "LOL": "Laugh out loud",
        "TBD": "To be determined",
        "IMHO/IMO": "In my humble opinion",
        "HMU": "Hit me up",
        "IIRC": "If I remember correctly",
        "LMK": "Let me know",
        "OG": "Original gangsters (used for old friends)",
        "FTW": "For the win",
        "NVM": "Nevermind",
        "OOTD": "Outfit of the day",
        "Ngl": "Not gonna lie",
        "Rq": "real quick",
        "Iykyk": "If you know, you know",
        "Ong": "On god (I swear)",
        "YAAAS": "Yes!",
        "Brt": "Be right there",
        "Sm": "So much",
        "Ig": "I guess",
        "Wya": "Where you at",
        "Istg": "I swear to god",
        "Hbu": "How about you",
        "Atm": "At the moment",
        "Asap": "As soon as possible",
        "Fyi": "For your information"
    }
    for word, expanded_form in chat_words.items():
        text = text.replace(word, expanded_form)
    return text

def remove_punctuation(text):
    clean_text = ''.join(ch for ch in text if ch not in string.punctuation)
    return clean_text

def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in newStopWords]
    return ' '.join(filtered_words)

def remove_whitespace(text):
    return text.strip()

def remove_special_characters(text):
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return clean_text


def preprocess_text(text):
    text = remove_html_tags(text)
    text = remove_urls(text)
    text = convert_to_lowercase(text)
    text = replace_chat_words(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = remove_whitespace(text)
    text = remove_special_characters(text)

    return text

df['text'] = df['text'].apply(preprocess_text)
df2['text'] = df2['text'].apply(preprocess_text)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-7-5bef1e73d4e7>:44: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
<ipython-input-7-5bef1e73d4e7>:44: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [ ]:

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=42)
# Vectorize text using TF-IDF

vectorizer = TfidfVectorizer(max_features=1000, ngram_range=(1,2), max_df=0.95, min_df=2 )

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


# K-means clustering
kmeans = KMeans(n_clusters=2, random_state=30)
kmeans.fit(X_train)
# Predict cluster labels
y_train_pred = kmeans.predict(X_train)
y_test_pred = kmeans.predict(X_test)
# Evaluate clustering performance
accuracy_train = f1_score(y_train, y_train_pred)
accuracy_test = f1_score(y_test, y_test_pred)
print("Training f1_score:", accuracy_train)
print("Test f1_score:", accuracy_test)

In [ ]:
inverse_ytrain_labels  = [1 if x==0 else 0 for x in y_train_pred]
inverse_ytest_labels  = [1 if x==0 else 0 for x in y_test_pred]
accuracy_train = f1_score(y_train, inverse_ytrain_labels)
accuracy_test = f1_score(y_test, inverse_ytest_labels)

print("Training f1_score inverse:", accuracy_train)
print("Test f1_score inverse", accuracy_test)

Training f1_score inverse: 0.652778469624157
Test f1_score inverse 0.6559970989933647


In [ ]:
from sklearn.cluster import Birch
from gensim.models import KeyedVectors
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler(with_mean=False)
X_scaled = scaler.fit_transform(X_train)
X_scled_test = scaler.fit_transform(X_test)

brc = Birch(branching_factor=50, n_clusters=2, threshold=0.5, compute_labels=True)


In [ ]:
brc.fit(X_scaled)

Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x7c7b43fc5750>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/nltk/data.py", line 1160, in __del__
    if not self.closed:
  File "/usr/local/lib/python3.10/dist-packages/nltk/data.py", line 1180, in closed
    return self.stream.closed
AttributeError: 'SeekableUnicodeStreamReader' object has no attribute 'stream'


In [ ]:
cluster_labels = brc.predict(X_scaled)

# osama

In [ ]:
from sklearn.cluster import KMeans

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['text'])

# K-means clustering
kmeans = KMeans(n_clusters=2, random_state=30)
kmeans.fit(X)

# Predict cluster labels
df['cluster'] = kmeans.labels_

# Transform the new dataset using the same vectorizer
X_test_new = vectorizer.transform(df2['text'])

# Predict clusters for the new dataset
predictions = kmeans.predict(X_test_new)
# Create a DataFrame for the predictions and save to CSV
results = pd.DataFrame({'id': test_ids, 'predicted_cluster': predictions})
results.to_csv('clust3.csv', index=False)

print(results.head())
